In [1]:
from ABDB import database as db
import matplotlib.pyplot as plt
import numpy as np
from retrain_ablooper import *
import torch
import pandas as pd
from rich import print as pprint

/Users/fabian/miniconda3/envs/auto-db-pipeline/lib/python3.9/site-packages/Bio/SubsMat/__init__.py:126: BiopythonDeprecationWarning: Bio.SubsMat has been deprecated, and we intend to remove it in a future release of Biopython. As an alternative, please consider using Bio.Align.substitution_matrices as a replacement, and contact the Biopython developers if you still need the Bio.SubsMat module.
  warnings.warn(
Database path /Volumes/LaCie/sabdab-sabpred/data/ABDB was not found.


In [ ]:
# torch settings
device = "cuda" if torch.cuda.is_available() else "cpu"
torch.set_default_dtype(torch.float)

model = MaskDecoyGen(decoys=1).to(device = device).float()
model.load_state_dict(torch.load("best_models/best_model-3004-Radam-1-2optim",
                                 map_location=torch.device(device)))

In [ ]:
from ABlooper import CDR_Predictor

for val in validation:

    pdb_id = val['ids'][0]
    heavy_c = val['ids'][1]['HC']
    light_c = val['ids'][1]['LC']

    rmsds_refined = []
    rmsds = []

    path = db.fetch("1gig").filepath
    input_path = "/".join(path.split("/")[:-1] + ["imgt"] + path.split("/")[-1:])
    output_path1 = pdb_id+heavy_c+light_c+'relaxed.pdb'
    output_path2 = pdb_id+heavy_c+light_c+'.pdb'

    pred = CDR_Predictor(input_path, chains = (heavy_c, light_c), model=model, refine=True)
    pred.write_predictions_in_pdb_format(output_path1)
    rmsds_refined.append(pred.calculate_BB_rmsd_wrt_input())
    
    pred = CDR_Predictor(input_path, chains = (heavy_c, light_c), model=model, refine=False)
    pred.write_predictions_in_pdb_format(output_path2)
    rmsds.append(pred.calculate_BB_rmsd_wrt_input())

    break

In [ ]:
from ABlooper import CDR_Predictor


input_path = "my_antibody_model.pdb"
output_path = "ABlooper_model.pdb"

pred = CDR_Predictor(input_path, chains = ("H", "L"))
pred.write_predictions_in_pdb_format(output_path)

rmsd_from_input = pred.calculate_BB_rmsd_wrt_input()
confidence_score = pred.decoy_diversity 